In [ ]:
%load_ext jupyflink.magics

In [ ]:
%flink_gateway_init --debug

In [ ]:
%%flink_gateway_sql_prepare
ADD JAR '/opt/gateway/jars/flink-sql-connector-kafka-1.16.2.jar';

In [ ]:
%%flink_gateway_sql_prepare
drop table if exists attributes;
CREATE TABLE attributes (
  `id` STRING,
  `entityId` STRING,
  `name` STRING,
  `nodeType` STRING,
  `valueType` STRING,
  `index` INTEGER,
  `type` STRING,
  `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
  `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
  `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
  `ts` TIMESTAMP(3) METADATA FROM 'timestamp' VIRTUAL,
  WATERMARK FOR ts AS ts
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild.attributes',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
);
drop table if exists attributes;
CREATE TABLE `attributes` (
    `id` STRING,
    `entityId` STRING,
    `name` STRING,
    `nodeType` STRING,
    `valueType` STRING,
    `index` INTEGER,
    `type` STRING,
    `https://uri.etsi.org/ngsi-ld/datasetId` STRING,
    `https://uri.etsi.org/ngsi-ld/hasValue` STRING,
    `https://uri.etsi.org/ngsi-ld/hasObject` STRING,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp' VIRTUAL,
        watermark FOR ts AS ts
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.ngsild.attributes'
);
DROP VIEW IF EXISTS `attributes_view`;
CREATE VIEW `attributes_view` AS
SELECT `id`,`type`,
`entityId`,
`name`,
`nodeType`,
`valueType`,
index,
`https://uri.etsi.org/ngsi-ld/datasetId`,
`https://uri.etsi.org/ngsi-ld/hasValue`,
`https://uri.etsi.org/ngsi-ld/hasObject`,
`ts` FROM (
  SELECT *,
ROW_NUMBER() OVER (PARTITION BY `id`,index
ORDER BY ts DESC) AS rownum
FROM `attributes` )
WHERE rownum = 1;
DROP VIEW IF EXISTS `attributes_view`;
CREATE VIEW `attributes_view` AS
SELECT
    id,
    entityId,
    name,
    nodeType,
    valueType,
    index,
    `type`,
     `https://uri.etsi.org/ngsi-ld/datasetId`,
     `https://uri.etsi.org/ngsi-ld/hasValue`,
     `https://uri.etsi.org/ngsi-ld/hasObject`,
     `ts` FROM (
        SELECT
            *,
             ROW_NUMBER() OVER (
                PARTITION BY `id`,
                `index` 
                ORDER BY
                    ts DESC
            ) AS rownum 
        FROM
            `attributes`
    ) 
WHERE
    rownum = 1
    and entityId is NOT NULL;

In [ ]:
%%flink_gateway_sql_prepare
drop table if exists ngsild_updates;
CREATE TABLE ngsild_updates (
  `op` STRING,
  `overwriteOrReplace` Boolean,
  `noForward` Boolean,
  `entities` STRING
) WITH (
  'connector' = 'kafka',
  'topic' = 'iff.ngsild-updates',
  'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
  'scan.startup.mode' = 'latest-offset',
  'format' = 'json'
);

In [ ]:
%%flink_gateway_sql_prepare
DROP TABLE IF EXISTS `alerts`;
CREATE TABLE `alerts` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY < STRING >,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,
    PRIMARY KEY (resource, event) NOT ENFORCED
) WITH (
    'connector' = 'kafka',
    'format' = 'json',
    'json.fail-on-missing-field' = 'False',
    'json.ignore-parse-errors' = 'True',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'scan.startup.mode' = 'latest-offset',
    'topic' = 'iff.alerts'
);

DROP TABLE IF EXISTS `alerts_bulk`;
CREATE TABLE `alerts_bulk` (
    `resource` STRING,
    `event` STRING,
    `environment` STRING,
    `service` ARRAY < STRING >,
    `severity` STRING,
    `customer` STRING,
    `text` STRING,
    watermark FOR ts AS ts - INTERVAL '0.0' SECONDS,
    `ts` TIMESTAMP(3) METADATA
    FROM
        'timestamp' VIRTUAL,
        PRIMARY KEY (resource, event) NOT ENFORCED
) WITH (
    'connector' = 'upsert-kafka',
    'value.format' = 'json',
    'value.json.fail-on-missing-field' = 'False',
    'value.json.ignore-parse-errors' = 'True',
    'key.format' = 'json',
    'properties.bootstrap.servers' = 'my-cluster-kafka-bootstrap:9092',
    'topic' = 'iff.alerts.bulk'
);

DROP VIEW IF EXISTS `alerts_bulk_view`;

